<a href="https://colab.research.google.com/github/ensemble-on/ChatterBot/blob/master/Copy_of_8_%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 사전을 이용한 감성분석

In [1]:
# from google.colab import auth
# auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
# 형태소분석기 관련 설치
!apt-get update
!apt-get install g++ openjdk-8-jdk

!pip install JPype1==0.7.0
!pip install rhinoMorph

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:5 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [777 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [682 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [5,665 B]
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic/main amd64 Packages [31.7 kB]
Get:13

In [9]:
# 경로 변경
cd /content/gdrive/My Drive/AIBM/20191016/pytest/

/content/gdrive/My Drive/AIBM/20191016/pytest


In [10]:
# 형태분석된 데이터 로딩
def read_data(filename, encoding='cp949'):               # 읽기 함수 정의
  with open(filename, 'r', encoding=encoding) as f:
    data = [line.split('\t') for line in f.read().splitlines()]
    data = data[1:]                                      # txt 파일의 헤더(id document label)는 제외하기
  return data

def write_data(data, filename, encoding='cp949'):        # 쓰기 함수 정의
  with open(filename, 'w', encoding=encoding) as f:
    f.write(data)

data = read_data('ratings_morphed.txt', encoding='cp949')

print(len(data))
print(len(data[0])) 
print(data[0])

492
3
['8132799', '디자인 배우 학생 외국 디자이너 일군 전통 통하 발전하 문화 산업 부럽 사실 우리나라 그 어렵 시절 끝 열정 지키 노라노 같 전통 있 같 사람 꿈 꾸 이루 나가 감사하', '1']


In [0]:
# 감정사전 읽기
# 직접 작성
data_id = [line[0] for line in data]       # 데이터 id
data_text = [line[1] for line in data]     # 데이터 본문 
data_senti = [line[2] for line in data]    # 데이터 긍부정 부분

positive = read_data('positive.txt') # 긍정 감정사전 읽기 
negative = read_data('negative.txt') # 부정 감정사전 읽기

pos_found = [] 
neg_found = []

In [13]:
# 감정단어 파악

def cntWordInLine(data, senti):
  senti_found = []
  for onedata in data:
    oneline_word = onedata.split(' ')       # 한 줄의 데이터를 공백 단위로 분리하여 리스트로 저장
    senti_temp = 0
    for sentiword in senti:
      if sentiword[0] in oneline_word:      # posword[0] 하여 리스트를 문자열로 추출
        senti_temp += 1                     # 현재의 감정단어와 일치하면 숫자를 하나 올려 줌 (중복은 계산 안 함)
    senti_found.append(senti_temp)          # 현재의 줄에서 찾은 감성단어의 숫자를 해당 위치에 저장
  return senti_found
      
      
data_senti_poscnt = cntWordInLine(data_text, positive)      # 발견된 긍정 단어의 숫자 파악
data_senti_negcnt = cntWordInLine(data_text, negative)      # 발견된 부정 단어의 숫자 파악

print(data_senti_poscnt[:20])
print(data_senti_negcnt[:20])

[5, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0, 1, 2, 0, 1, 0, 1, 0]
[1, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1]


In [0]:
# 감정점수 계산
# Pandas 데이터프레임으로 저장
import pandas as pd
newdata = pd.DataFrame({'id':data_id, 'text':data_text, 'original':data_senti, 'pos':data_senti_poscnt, 'neg':data_senti_negcnt})
senti_score = newdata['pos'] - newdata['neg']      # 긍정개수에서 부정개수를 뺌
newdata['senti_score'] = senti_score               # 그 수를 senti_score 컬럼에 저장 

newdata.loc[newdata.senti_score > 0, 'new'] = 1    # 새로운 긍부정 기호
newdata.loc[newdata.senti_score <= 0, 'new'] = 0   # 새로운 긍부정 기호

# 처음에 기록된 긍부정과 새로 계산된 긍부정이 같은지 여부를 matched 컬럼에 저장
# original 컬럼은 문자로 되어 있으므로 숫자로 변환 뒤 비교 
# 직접 작성
newdata.loc[pd.to_numeric(newdata.original) == newdata.new, 'matched'] = 'True' 
newdata.loc[pd.to_numeric(newdata.original) != newdata.new, 'matched'] = 'False'

In [15]:
# 원점수와 비교 및 저장
# 직접 작성

score = newdata.matched.str.count('True').sum() / (newdata.matched.str.count('True').sum() 
                                  + newdata.matched.str.count('False').sum()) * 100 
print(score)
newdata.to_csv('newfile.csv', sep=',', encoding='cp949', index=False) # csv 저장 
newdata.to_csv('newfile2.txt', sep='\t', encoding='cp949', index=False) # 또는 txt 저장


65.2439024390244


In [0]:
# 시그모이드 점수 계산
import math
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

newdata['sigmoid'] = newdata.senti_score.apply(sigmoid)

In [17]:
# 결과 확인
# 직접 작성
pd.set_option('display.max_columns', 10)     # 결과 컬럼 확장
print(newdata.iloc[:5, :])

         id                                               text original  pos  \
0   8132799  디자인 배우 학생 외국 디자이너 일군 전통 통하 발전하 문화 산업 부럽 사실 우리나...        1    5   
1   4655635                                  폴리스스토리 시리즈 뉴 없 최고        1    1   
2   9251303               와 연기 진짜 쩔 지루 하 생각하 몰입하 보 그래 이런 진짜 영화        1    0   
3  10067386                             안개 자욱 하 밤하늘 뜨 초승달 같 영화        1    0   
4   2190435                                 사랑 해보 사람 처음 끝 웃 영화        1    2   

   neg  senti_score  new matched   sigmoid  
0    1            4  1.0    True  0.982014  
1    0            1  1.0    True  0.731059  
2    1           -1  0.0   False  0.268941  
3    0            0  0.0   False  0.500000  
4    0            2  1.0    True  0.880797  


# 머신러닝을 이용한 감성분석

In [18]:
# 형태분석된 데이터 로딩
def read_data(filename, encoding='cp949'):               # 읽기 함수 정의
  with open(filename, 'r', encoding=encoding) as f:
    data = [line.split('\t') for line in f.read().splitlines()]
    data = data[1:]                                      # txt 파일의 헤더(id document label)는 제외하기
  return data

def write_data(data, filename, encoding='cp949'):        # 쓰기 함수 정의
  with open(filename, 'w', encoding=encoding) as f:
    f.write(data)

data = read_data('ratings_morphed.txt', encoding='cp949')

print(len(data))
print(len(data[0])) 
print(data[0])

492
3
['8132799', '디자인 배우 학생 외국 디자이너 일군 전통 통하 발전하 문화 산업 부럽 사실 우리나라 그 어렵 시절 끝 열정 지키 노라노 같 전통 있 같 사람 꿈 꾸 이루 나가 감사하', '1']


In [24]:
# 훈련데이터와 테스트데이터 분리 (자동)
# 직접 작성
data_text = [line[1] for line in data]  # 데이터 본문
data_senti = [line[2] for line in data] # 데이터 긍부정 부분

from sklearn.model_selection import train_test_split
train_data_text, test_data_text, train_data_senti, test_data_senti = train_test_split(data_text, data_senti, stratify=data_senti)

# Counter 클래스를 이용해 각 분류가 훈련데이터와 테스트데이터에 같은 비율로 들어갔는지 확인해 본다 
from collections import Counter
train_data_senti_freq = Counter(train_data_senti)
print('train_data_senti_freq:', train_data_senti_freq)

test_data_senti_freq = Counter(test_data_senti)
print('test_data_senti_freq:', test_data_senti_freq)

train_data_senti_freq: Counter({'0': 185, '1': 184})
test_data_senti_freq: Counter({'0': 62, '1': 61})


In [26]:
# 훈련데이터와 테스트데이터 분리 (수동)
# 직접 작성

import random 
random.shuffle(data)                            # 랜덤하게 섞는다

data_70 = int(len(data)*0.7) # 전체 데이터 크기의 70% 숫자를 찾는다 
train_data = data[:data_70] # 앞에서 70% 부분을 잘라 훈련데이터로 
test_data = data[data_70:] # 그 뒷부분을 테스트데이터로

print('train data length:', len(train_data))    # 138212 
print('test data length:', len(test_data))      # 59235

# 훈련데이터 요소 분리 
train_data_text = [line[1] for line in train_data]      # 훈련데이터 본문 
train_data_senti = [line[2] for line in train_data]     # 훈련데이터 긍부정 부분

# 테스트데이터 요소 분리 
test_data_text = [line[1] for line in test_data]        # 테스트데이터 본문 
test_data_senti = [line[2] for line in test_data]       # 테스트데이터 긍부정 부분

train data length: 344
test data length: 148


In [29]:
# 행렬 형태로 변환
# 직접 작성

from sklearn.feature_extraction.text import CountVectorizer 
vect = CountVectorizer(min_df=5).fit(train_data_text) # 빈도 5이상의 단어만 대상 
X_train = vect.transform(train_data_text) # 행렬 생성 
print("X_train:\n", repr(X_train))

X_train:
 <344x58 sparse matrix of type '<class 'numpy.int64'>'
	with 623 stored elements in Compressed Sparse Row format>


In [30]:
# 행렬 내용 관찰
# 직접 작성
feature_names = vect.get_feature_names() 
print("특성 개수:", len(feature_names)) 
print("처음 20개 특성:\m", feature_names[:20]) 
print("3000~5000까지의 특성:\n", feature_names[3000:5000])

특성 개수: 58
처음 20개 특성:\m ['ㅋㅋ', 'ㅋㅋㅋ', 'ㅠㅠ', 'ㅡㅡ', '가지', '감독', '감동', '그렇', '나오', '남자', '내내', '내용', '너무', '다시', '드라마', '마음', '마지막', '만들', '명작', '모르']
3000~5000까지의 특성:
 []


In [31]:
# 머신러닝 알고리즘 적용
# 직접 작성
import pandas as pd 
from sklearn.model_selection import cross_val_score 
from sklearn.linear_model import LogisticRegression 
y_train = pd.Series(train_data_senti) # 리스트 형태를 종속변수가 될 수 있는 1차원 배열(Series)로 만든다

lr = LogisticRegression(solver="liblinear")# 모델 생성 
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [32]:
# 테스트 데이터 입력
X_test = vect.transform(test_data_text)
y_test = pd.Series(test_data_senti)

print("테스트 데이터 점수:", lr.score(X_test, y_test))

테스트 데이터 점수: 0.6418918918918919


# 1개 데이터 예측


In [33]:
# 형태소분석기 시작
import rhinoMorph as rhinoMorph
rn = rhinoMorph.startRhino()

# 형태소 분석
new_input = '오늘은 정말 재미있는 하루구나!'
inputdata = []
morphed_input = rhinoMorph.onlyMorph_list(
    rn, new_input, pos=['NNG', 'NNP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ'])
morphed_input = ' '.join(morphed_input) # ['오늘', '정말', '재미있', '하루＇]를 한 개 문자열로 변환

inputdata.append(morphed_input)         # 분석 결과를 리스트로 만들기
print('input data:', inputdata)         # ['오늘 정말 재미있 하루']

filepath:  /usr/local/lib/python3.6/dist-packages
classpath:  /usr/local/lib/python3.6/dist-packages/rhinoMorph/lib/rhino.jar


/usr/local/lib/python3.6/dist-packages/jpype/_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


RHINO started!
input data: ['오늘 정말 재미있 하루']


In [34]:
X_input = vect.transform(inputdata)	    # 앞에서 만든 11445 컬럼의 행렬에 적용
result = lr.predict(X_input) 	          # 0은 부정,1은 긍정

if result == 0:
  print("부정적인 글입니다")
else:
  print("긍정적인 글입니다")

긍정적인 글입니다
